<a href="https://colab.research.google.com/github/yimingsun12138/IDE_on_Colab/blob/main/Web_IDE_on_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check Google Colab IP location
Re-connect the Colab server if you do not like the IP location.

In [ ]:
!curl getip.mrdoge.cool

# Global setting

In [ ]:
#@title Global setting

#@markdown Choose web IDE and set user info.

web_IDE = 'Rstudio server' #@param ["Rstudio server", "code server"]
user_name = 'Knight' #@param {type:"string"}
user_password = 'midnight' #@param {type:"string"}

#@markdown Default using localtunnel for tunnelling.

use_localtunnel = True #@param {type:"boolean"}

#@markdown Leave the parameters alone if you do not choose code server.

code_server_port = '8585' #@param {type:"string"}
code_server_password = 'evernight' #@param {type:"string"}

# Install web IDE

## Update environment

In [ ]:
!apt update
!apt upgrade
!apt autoremove

## Clone source code

In [ ]:
!git clone https://github.com/yimingsun12138/IDE_on_Colab.git

## Add user

In [ ]:
import os
os.system("useradd -m -s /bin/bash " + user_name)
os.system("echo '" + user_name + ":" + user_password + "' | chpasswd")
os.system("usermod -G sudo " + user_name)

## Install dependency

In [ ]:
!apt install expect

## Install Rstudio server

In [ ]:
if web_IDE == 'Rstudio server':
  os.system("bash /content/IDE_on_Colab/Install_Rstudio_server.sh")
  tunnel_port = '8787'

## Install code server

In [ ]:
if web_IDE == 'code server':
  os.system("expect /content/IDE_on_Colab/Install_code_server.exp" + " " + user_name + " " + user_password + " " + code_server_port + " " + code_server_password)
  tunnel_port = code_server_port

# Install localtunnel

In [ ]:
!touch /content/IDE_on_Colab/localtunnel.log
if use_localtunnel:
  os.system("npm install -g localtunnel")
  os.system("nohup lt --port " + tunnel_port + " > /content/IDE_on_Colab/localtunnel.log 2>&1 &")
!sleep 10
!cat /content/IDE_on_Colab/localtunnel.log | grep "your url is:"

# Keep Colab session alive

In [ ]:
if use_localtunnel:
  import numpy as np
  import time
  ii = 1

  while True:
    i = np.random.rand(100,100)
    i = np.dot(i,i)
    i = 'round ' + str(ii)
    print(i)
    ii = ii + 1
    time.sleep(60)
else:
  print("Set your own tunnel below!")